# Project 1 “What causes antibiotic resistance?” Alignment to reference, variant calling.


#### 27.10.2024

Today I start working on project #1 connected with searching mutations(SNPs) in _E.Coli_ _K12 MG1655_ genome. During this work basic steps like quality control, alignment to reference and variant calling will be demonstrated.

Our input data we are going to analyze:
1) Reference sequence of the parental (unevolved, not resistant to antibiotics) [E. coli strain.](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/) Here sequence in fasta format (GCF_000005845.2_ASM584v2_genomic.fna.gz) and annotation in .gff format(*_genomic.gff.gz).
2)  This is E.coli strain K-12 substrain MG1655. More information about this genome can be [here](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000005845.2/).
3) Raw Illumina sequencing reads from shotgun sequencing of an [E. coli](https://doi.org/10.6084/m9.figshare.10006541.v3) strain that is resistant to the antibiotic ampicillin: (1 and 2 refer to forward and reverse, this was a paired end run).



For this project many different tools need to be installed: `seqkit`, `fastqc`, `trimmomatic`, `bwa`, `samtools`, `varscan`, `snpeff`. Let's start with instllation all of them.

In [6]:
!conda install -c bioconda seqkit # allows to get statistics for raw reads
!conda install bioconda::fastqc # fastqc makes  an overview of qulity of reads
!conda install bioconda::trimmomatic # trimmomatic is necessary to remove adapters, remove bad quality reeds
!conda install bwa # tool for alignment to reference sequence
!conda install bioconda::samtools # allows to proccess data in SAM, BAM, mpileup formats
!conda install bioconda::varscan # variant calling
!conda install bioconda::snpeff # automaticcaly variants annotation

First take a look at our data first, using command head:

In [7]:
!head -20 GCF_000005845.2_ASM584v2_genomic.fna 


In [8]:
!cat GCF_000005845.2_ASM584v2_genomic.gff | head -10


Also let’s count the original amount of lines in fastq files to understand initial amount of reads:

In [13]:
!wc -l amp_res_*

  1823504 amp_res_1.fastq
  1823504 amp_res_2.fastq
  3647008 итого


In [16]:
print(f'Total amount of reads:{1823504/4}')


Total amount of reads:455876.0


In addition to manual inspection, we may use tool `seqkit` to get detailed statistics about equencing data.


In [20]:
!seqkit stats amp_res_1.fastq
!seqkit stats amp_res_2.fastq

file             format  type  num_seqs     sum_len  min_len  avg_len  max_len
amp_res_1.fastq  FASTQ   DNA    455,876  46,043,476      101      101      101
file             format  type  num_seqs     sum_len  min_len  avg_len  max_len
amp_res_2.fastq  FASTQ   DNA    455,876  46,043,476      101      101      101


## Data quality control

After downloading raw sequences we may make a quick impression of whether our data has any problems of which we should be aware before doing any further analysis. FastQc allows to do some quality control checks on raw sequence data.

In [22]:
! fastqc -h # Check that programm is working and properly installed

First, create a directory for FastQC reports and than run command.

In [25]:
!mkdir FastQC
!fastqc -o FastQC amp_res_1.fastq amp_res_2.fastq 

null
null
Started analysis of amp_res_1.fastq
Approx 5% complete for amp_res_1.fastq
Approx 10% complete for amp_res_1.fastq
Approx 15% complete for amp_res_1.fastq
Approx 20% complete for amp_res_1.fastq
Approx 25% complete for amp_res_1.fastq
Approx 30% complete for amp_res_1.fastq
Approx 35% complete for amp_res_1.fastq
Approx 40% complete for amp_res_1.fastq
Approx 45% complete for amp_res_1.fastq
Approx 50% complete for amp_res_1.fastq
Approx 55% complete for amp_res_1.fastq
Approx 60% complete for amp_res_1.fastq
Approx 65% complete for amp_res_1.fastq
Approx 70% complete for amp_res_1.fastq
Approx 75% complete for amp_res_1.fastq
Approx 80% complete for amp_res_1.fastq
Approx 85% complete for amp_res_1.fastq
Approx 90% complete for amp_res_1.fastq
Approx 95% complete for amp_res_1.fastq
Analysis complete for amp_res_1.fastq
Started analysis of amp_res_2.fastq
Approx 5% complete for amp_res_2.fastq
Approx 10% complete for amp_res_2.fastq
Approx 15% complete for amp_res_2.fastq
Ap

We open the report for forward and reverse readings and observe the following results:
1. The number of rows is 455876, which coincides with the previously calculated values
2.  Several sections have not passed quality control and have a red(very unusual) or yellow(slightly abnormal) designation:
   - Per base sequence quality
   - Per tile sequence quality
   - Per base sequence content
   - Per sequence GC content

A few words about this plots:

**Per base sequence quality**:  provides the distribution of quality scores at each position in the read across all reads.

According to documentation a warning will be issued if the lower quartile for any base is less than 10, or if the median for any base is less than 25. We can easily see that by the end the quality of the reads is noticeably decreasing, especially the 96-97 positions.
The main reason for the decreasing sequence quality is the so-called phasing that means that the blocker of a nucleotide is not correctly removed after signal detection.

**Per tile sequence quality**:  this plot enables to look at the quality scores from each tile across all of your bases to see if there was a loss in quality associated with only one part of the flowcell

**Per base sequence content**:  provides the proportion of each base position in a file for which each of the four normal DNA bases has been called.

**Per sequence GC content**:this module measures the GC content across the whole length of each sequence in a file and compares it to a modelled normal distribution of GC content.

To improve the quality of our data, we will use `TrimmomaticPE` to trim technical sequences and remove low-quality reads.  Let's set the values of the parameters by which the readings will be filtered:

      - ILLUMINACLIP:NexteraPE-PE.fa:2:30:10:2. NexteraPE-PE.fa - file with Nextera Transposase sequences 
      - LEADING:20,  TRAILING:20 
      - SLIDINGWINDOW:10:20 - a sliding window method for evaluating the quality of sequences. 
      - MINLEN:20 - the min length of reads

In [16]:
!mkdir TrimmomaticReads
# Run Trimmomatic
!trimmomatic PE -phred33 ./amp_res_1.fastq ./amp_res_2.fastq \
    ./TrimmomaticReads/output_1_paired.fq.gz ./TrimmomaticReads/output_1_unpaired.fq.gz \
    ./TrimmomaticReads/output_2_paired.fq.gz ./TrimmomaticReads/output_2_unpaired.fq.gz \
    ILLUMINACLIP:./NexteraPE-PE.fa:2:30:10:2:True \
    LEADING:20 SLIDINGWINDOW:10:20 TRAILING:20 MINLEN:20


mkdir: невозможно создать каталог «TrimmomaticReads»: Файл существует
TrimmomaticPE: Started with arguments:
 -phred33 ./amp_res_1.fastq ./amp_res_2.fastq ./TrimmomaticReads/output_1_paired.fq.gz ./TrimmomaticReads/output_1_unpaired.fq.gz ./TrimmomaticReads/output_2_paired.fq.gz ./TrimmomaticReads/output_2_unpaired.fq.gz ILLUMINACLIP:./NexteraPE-PE.fa:2:30:10:2:True LEADING:20 SLIDINGWINDOW:10:20 TRAILING:20 MINLEN:20
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Input Read Pairs: 455876 Both Surviving: 446249 (97,89%) Forward Only Surviving: 9218 (2,02%) Reverse Only Surviving: 281 (0,06%) 

Here we can see infromation how much reads passed filtering (both, forward only, reverse only surviving) and amount of dropped reads. We wiil continue working only with reads where both strands survived. After trimming we can check how the quality of data was changed. As we gonna work only with paired reads run fastqc only for them again.

In [21]:
!fastqc ./TrimmomaticReads/output_1_paired.fq.gz ./TrimmomaticReads/output_2_paired.fq.gz -o ./FastQC

application/gzip
application/gzip
Started analysis of output_1_paired.fq.gz
Approx 5% complete for output_1_paired.fq.gz
Approx 10% complete for output_1_paired.fq.gz
Approx 15% complete for output_1_paired.fq.gz
Approx 20% complete for output_1_paired.fq.gz
Approx 25% complete for output_1_paired.fq.gz
Approx 30% complete for output_1_paired.fq.gz
Approx 35% complete for output_1_paired.fq.gz
Approx 40% complete for output_1_paired.fq.gz
Approx 45% complete for output_1_paired.fq.gz
Approx 50% complete for output_1_paired.fq.gz
Approx 55% complete for output_1_paired.fq.gz
Approx 60% complete for output_1_paired.fq.gz
Approx 65% complete for output_1_paired.fq.gz
Approx 70% complete for output_1_paired.fq.gz
Approx 75% complete for output_1_paired.fq.gz
Approx 80% complete for output_1_paired.fq.gz
Approx 85% complete for output_1_paired.fq.gz
Approx 90% complete for output_1_paired.fq.gz
Approx 95% complete for output_1_paired.fq.gz
Analysis complete for output_1_paired.fq.gz
Started

Now let's check that everything is right and we have what we see.

In [22]:
!for file in ./TrimmomaticReads/output_1_paired.fq.gz ./TrimmomaticReads/output_2_paired.fq.gz; do \
    echo "$file: $(zcat "$file" | wc -l | awk '{print $1 / 4}') ридов"; \
done


./TrimmomaticReads/output_1_paired.fq.gz: 446249 ридов
./TrimmomaticReads/output_2_paired.fq.gz: 446249 ридов


Alright, everything seems good. Now we continue with alignment our reads to reference genome.



## Alignment

First, it's necessary to index reference genome to make alignment process faster. After this wi will use command `bwa-mem` for mapping our reads

In [37]:
%%capture
# index reference file
!bwa index GCF_000005845.2_ASM584v2_genomic.fna 
# align trimmed reads to reference
!bwa mem GCF_000005845.2_ASM584v2_genomic.fna ./TrimmomaticReads/output_1_paired.fq.gz ./TrimmomaticReads/output_2_paired.fq.gz > ./alignment/alignment.sam


[bwa_index] Pack FASTA... 0.05 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.43 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.50 sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa index GCF_000005845.2_ASM584v2_genomic.fna
[main] Real time: 2.069 sec; CPU: 2.053 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 107740 sequences (10000093 bp)...
[M::process] read 109660 sequences (10000066 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (10, 51910, 0, 22)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (91, 136, 310)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 748)
[M::mem_pestat] mean and std.dev: (155.22, 100.95)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 967)
[M::mem_pestat] analyzing insert size distr

The SAM file contains all reads, whether they are successfully aligned or not. For bioinformatics pipelines, it’s important to know what fraction of your reads was aligned to estimate contaminations if any.
First, we need to compress and sort the SAM file with the commands below. A compressed SAM file is called a BAM file. 
BAM files need to be sorted and indexed. This indexing is different from the reference indexing. For BAM file indexing, given a coordinate-sorted bam file, you can pull reads from desired positions very quickly, instead of having to iterate over the entire file each time you look for a specific coordinate.

In [33]:
# compress SAM to BAM
!samtools view -S -b ./alignment/alignment.sam > ./alignment/alignment.bam


In [34]:
# sort BAM file by sequence coordinate on reference
!samtools sort ./alignment/alignment.bam -o ./alignment/alignment_sorted.bam


In [35]:
# index BAM file for faster search
!samtools index ./alignment/alignment_sorted.bam


We can get some basic statistics about alignment

In [39]:
!samtools flagstat ./alignment/alignment.bam

892755 + 0 in total (QC-passed reads + QC-failed reads)
892498 + 0 primary
0 + 0 secondary
257 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
891628 + 0 mapped (99.87% : N/A)
891371 + 0 primary mapped (99.87% : N/A)
892498 + 0 paired in sequencing
446249 + 0 read1
446249 + 0 read2
888532 + 0 properly paired (99.56% : N/A)
890392 + 0 with itself and mate mapped
979 + 0 singletons (0.11% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


All reads passed QC.
891628 reads (99.87%) were aligned to the reference genome, of which 888532
(99.56%) were correctly aligned, i.e. both sequences in a pair are located at the
expected distance and oriented in opposite directions. The share of single reads
was 0.11%.

## Variant calling

The goal now is to go through our data, and for each position in the reference, see how many reads have a mutation at the same position - we want to distinguish actual mutations frome sequencing errors. The solution is to make an intermediate file type called an mpileup, because it goes through each position and “piles up” the reads, tabulating the number of bases that match or don’t match the reference.


In [40]:
!samtools mpileup -f GCF_000005845.2_ASM584v2_genomic.fna ./alignment/alignment_sorted.bam >  ./variants/my.mpileup


[mpileup] 1 samples in 1 input files


To call actual variants, we will be using a program called VarScan. Run VarScan on the mpileup.

We set the `-min-var-freq` parameter to 50%, which mean the minimum percentage of
non-reference bases needed to determine the variant in the sample. 

`--variants` flag tells VarScan to only  output positions that are above our threshold, and `--output-vcf 1` option tells it we want the output in yet another kind of data format called vcf 

In [1]:
!varscan mpileup2snp ./variants/resistance.mpileup --min-var-freq 0.50 --variants --output-vcf 1 > ./variants/resistance_VarScan_0.5.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.5
Min avg qual:	15
P-value thresh:	0.01
Reading input from ./variants/resistance.mpileup
4641495 bases in pileup file
9 variant positions (6 SNP, 3 indel)
0 were failed by the strand-filter
6 variant positions reported (6 SNP, 0 indel)


Here we can see VarScan results. The program detected 6 SNPs.

We can analyze called variants using IGV browser or make automatically annotations of SNPs using snpEff utility withcommands below

In [8]:
# a) create empty text file snpEff.config, and add there just one string: k12.genome : ecoli_K12
!touch snpEff.config
!nano snpEff.config # k12.genome : ecoli_K12
# b) create folder for the database
!mkdir -p data/k12
c) Put there .gbk file (unzip and rename to genes.gbk)
!gunzip GCF_000005845.2_ASM584v2_genomic.gbff.gz
!cp GCF_000005845.2_ASM584v2_genomic.gbff data/k12/genes.gbk
d) create database and annotate
!snpEff build -genbank -v k12
!snpEff ann k12 resistance_VarScan_0.5.vcf > VarScan_results_annonated_0.5.vcf

After work is done snpEff makes .html report with all detected SNPs

Results:
![Here is summary information about SNPs](summary.png)
![Table codon changes](codons.png)
![Table amino acid changes](aminoacids.png)